# AI Travel Planner — MCP

**Requirements:** You'll need a Open Router API key.

## Install dependencies

In [2]:
!pip -q install --quiet smolagents google-generativeai ddgs fastapi uvicorn[standard] pydantic requests nest-asyncio
print('Installed dependencies')

Installed dependencies


## Configuration

Provide your Open Router API key.

In [9]:
import os
import sys
import warnings
import getpass
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') or getpass.getpass('Enter your Open Router API key: ').strip()
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
print('Gemini API key configured.')

Enter: ··········
Gemini API key configured.


In [14]:
# Imports
from ddgs import DDGS
import google.generativeai as genai
import time, json, re
from typing import List, Dict, Any
import requests

# Configure Gemini
genai.configure(api_key=os.environ.get('GEMINI_API_KEY'))

ddgs = DDGS()

def web_search(query: str, max_results: int = 5):
    """Perform a DuckDuckGo search and return a list of result dicts."""
    results = []
    for r in ddgs.text(query, region='wt-wt', safesearch='Off', timelimit='y'):
        # ddgs.text yields generator; break when we have enough
        results.append({'title': r.get('title'), 'body': r.get('body'), 'href': r.get('href')})
        if len(results) >= max_results:
            break
    return results

def llm_summarize(prompt: str, model: str = 'google/gemini-2.0-flash-exp:free', max_output_tokens: int = 512):
    """Call OpenRouter API to get a completion/synthesis."""

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {GEMINI_API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com/",
            "X-Title": "AI Travel Planner Colab Notebook",
        },
        data=json.dumps({
            "model": model,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "max_tokens": max_output_tokens
        })
    )

    try:
        response_json = response.json()
        if response_json and 'choices' in response_json and response_json['choices']:
            return response_json['choices'][0]['message']['content'].strip()
        else:
            print("Warning: Unexpected response structure from OpenRouter API.")
            print(response_json)
            return str(response_json)
    except Exception as e:
        print(f"Error parsing OpenRouter API response: {e}")
        print(response.text)
        return response.text.strip() if response.text else str(response)

## Agent functions

These functions perform live web searches and LLM synthesis to produce flight/hotel search results and itineraries.

In [15]:
from datetime import datetime, timedelta

def search_flights(origin: str, destination: str, depart_date: str, return_date: str = '', passengers: int = 1):
    """Search web for flights and synthesize results with Gemini."""
    query = f"flights from {origin} to {destination} {depart_date} {('return '+return_date) if return_date else ''} best fares"
    results = web_search(query, max_results=6)
    # Compose LLM prompt
    prompt = 'You are a travel assistant. Extract the most relevant flight options from these search snippets and present as JSON array with fields airline, price_estimate, depart, stops, link. Input snippets:\n\n' + json.dumps(results)
    summary = llm_summarize(prompt)
    # Try to extract JSON from LLM output
    json_blob = extract_json_from_text(summary)
    if json_blob:
        return json_blob
    else:
        return {'summary_text': summary, 'raw_snippets': results}

def search_hotels(destination: str, checkin: str, checkout: str, guests: int = 1):
    query = f"hotels in {destination} check-in {checkin} check-out {checkout} best deals"
    results = web_search(query, max_results=6)
    prompt = 'You are a travel assistant. From these snippets, extract top hotel options as JSON with fields name, nightly_price_estimate, rating, link. Snippets:\n\n' + json.dumps(results)
    summary = llm_summarize(prompt)
    json_blob = extract_json_from_text(summary)
    if json_blob:
        return json_blob
    else:
        return {'summary_text': summary, 'raw_snippets': results}

def plan_itinerary(destination: str, days: int, interests: List[str] = None):
    interests = interests or ['sightseeing','food','culture']
    query = f"top things to do in {destination} {days} day itinerary {', '.join(interests)}"
    results = web_search(query, max_results=8)
    prompt = 'You are a helpful travel planner. Based on these snippets, create a day-by-day itinerary for ' + str(days) + ' days focusing on ' + ','.join(interests) + '. Return JSON with keys: destination, days, itinerary (list of day objects with title and activities). Snippets:\n\n' + json.dumps(results)
    summary = llm_summarize(prompt, model='gemini-2.0-flash', max_output_tokens=800)
    json_blob = extract_json_from_text(summary)
    if json_blob:
        return json_blob
    else:
        return {'summary_text': summary, 'raw_snippets': results}

# Helper to extract JSON from LLM text (best-effort)
def extract_json_from_text(text: str):
    idx = min([i for i in [text.find('{'), text.find('[')] if i!=-1] or [-1])
    if idx == -1:
        return None
    candidate = text[idx:]
    for end in range(len(candidate), 0, -1):
        try:
            parsed = json.loads(candidate[:end])
            return parsed
        except Exception:
            continue
    m = re.search(r'({.*}|\[.*\])', text, flags=re.DOTALL)
    if m:
        try:
            return json.loads(m.group(1))
        except Exception:
            return None
    return None

## Example: Run agents locally

In [16]:
from datetime import datetime, timedelta

# Example usage
origin = input('Origin (e.g., Casablanca): ').strip() or 'Casablanca'
destination = input('Destination (e.g., Paris): ').strip() or 'Paris'
depart_date = input('Depart date (YYYY-MM-DD): ').strip() or '2025-10-10'
nights = int(input('Number of nights: ').strip() or '5')

print('Searching flights...')
flights = search_flights(origin, destination, depart_date)
print('\nFlights result:\n', json.dumps(flights, indent=2))

print('\nSearching hotels...')
# Calculate checkout date based on depart_date and nights
checkin_date = datetime.strptime(depart_date, '%Y-%m-%d')
checkout_date = checkin_date + timedelta(days=nights)
checkout_date_str = checkout_date.strftime('%Y-%m-%d')

hotels = search_hotels(destination, depart_date, checkout_date_str)
print('\nHotels result:\n', json.dumps(hotels, indent=2))

print('\nPlanning itinerary...')
itinerary = plan_itinerary(destination, nights, interests=['museums','food'])
print('\nItinerary result:\n', json.dumps(itinerary, indent=2))

Origin (e.g., SFO): rabat
Destination (e.g., Paris): tokyo
Depart date (YYYY-MM-DD): 2025-10-10
Number of nights: 7
Searching flights...
{'error': {'message': 'No auth credentials found', 'code': 401}}

Flights result:
 {
  "summary_text": "{'error': {'message': 'No auth credentials found', 'code': 401}}",
  "raw_snippets": [
    {
      "title": "Cheap Flights from Chicago to Rabat",
      "body": "... flight cost data from across the web for ... Increased flexibility is the main benefit when it comes to buying a one way flight from Chicago to Rabat .",
      "href": "https://www.farecompare.com/flights/Chicago-CHI/Rabat-RBA/market.html"
    },
    {
      "title": "Cheap flights Paris - Rabat, Morocco - Trabber",
      "body": "The airlines with direct flights from Paris to Rabat are ... To get the best price from Paris to Rabat it is recommended to book 90 days in advance.",
      "href": "https://www.trabber.us/flights-paris-rabat-par-rba/"
    },
    {
      "title": "\u20ac62 Che

## MCP Server

This cell implements a FastAPI-based MCP-like server that exposes `list_resources`, `list_tools`, and `call_tool`.

In [17]:

# MCP server implementation (FastAPI)
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Any
import threading, time, requests, nest_asyncio, uvicorn

nest_asyncio.apply()

app = FastAPI(title='Notebook MCP Server (Real Agents)')

class Resource(BaseModel):
    uri: str
    name: str
    mimeType: str
    description: str = ''

class Tool(BaseModel):
    name: str
    description: str
    inputSchema: Dict[str, Any] = {}

# In-memory registries
RESOURCES: List[Resource] = []
TOOLS: List[Tool] = []

def register_real_tools():
    RESOURCES.clear(); TOOLS.clear()
    RESOURCES.extend([
        Resource(uri='travel://flights/search', name='Flight Search Results', mimeType='application/json'),
        Resource(uri='travel://hotels/search', name='Hotel Search Results', mimeType='application/json'),
        Resource(uri='travel://itineraries/latest', name='Latest Itinerary', mimeType='application/json')
    ])
    TOOLS.extend([
        Tool(name='search_flights', description='Search flights via web + LLM', inputSchema={'origin':'string','destination':'string','depart_date':'string','return_date':'string?','passengers':'int?'}),
        Tool(name='search_hotels', description='Search hotels via web + LLM', inputSchema={'destination':'string','checkin':'string','checkout':'string','guests':'int?'}),
        Tool(name='plan_itinerary', description='Plan itinerary via web + LLM', inputSchema={'destination':'string','days':'int','interests':'array?'})
    ])

register_real_tools()

@app.get('/list_resources', response_model=List[Resource])
def list_resources():
    return RESOURCES

@app.get('/list_tools', response_model=List[Tool])
def list_tools():
    return TOOLS

class CallToolReq(BaseModel):
    name: str
    arguments: Dict[str, Any] = {}

@app.post('/call_tool')
def call_tool(req: CallToolReq):
    name = req.name
    args = req.arguments or {}
    if name == 'search_flights':
        out = search_flights(**args)
        # store in resource description as JSON string
        for r in RESOURCES:
            if r.uri == 'travel://flights/search':
                r.description = json.dumps(out)
        return {'status':'ok','result': out}
    elif name == 'search_hotels':
        out = search_hotels(**args)
        for r in RESOURCES:
            if r.uri == 'travel://hotels/search':
                r.description = json.dumps(out)
        return {'status':'ok','result': out}
    elif name == 'plan_itinerary':
        out = plan_itinerary(**args)
        for r in RESOURCES:
            if r.uri == 'travel://itineraries/latest':
                r.description = json.dumps(out)
        return {'status':'ok','result': out}
    else:
        raise HTTPException(status_code=404, detail=f'Tool {name} not found')

def run_server_bg(host='127.0.0.1', port=8000):
    config = uvicorn.Config(app, host=host, port=port, log_level='info', loop='asyncio')
    server = uvicorn.Server(config)
    thread = threading.Thread(target=server.run, daemon=True)
    thread.start()
    time.sleep(0.8)
    return thread

print('MCP server ready. Call run_server_bg() to start.')

MCP server ready. Call run_server_bg() to start.


## Start MCP server (background)

Run this cell to start the server; then use the example client cell to call tools remotely.

In [18]:

# Start server
thread = run_server_bg(host='127.0.0.1', port=8000)
print('Server thread:', thread.name)
# show tools/resources
print('\nTools:')
for t in TOOLS:
    print('-', t.name, ':', t.description)
print('\nResources:')
for r in RESOURCES:
    print('-', r.uri, ':', r.name)

INFO:     Started server process [1674]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Server thread: Thread-4 (run)

Tools:
- search_flights : Search flights via web + LLM
- search_hotels : Search hotels via web + LLM
- plan_itinerary : Plan itinerary via web + LLM

Resources:
- travel://flights/search : Flight Search Results
- travel://hotels/search : Hotel Search Results
- travel://itineraries/latest : Latest Itinerary


## Example: MCP client (calls server endpoints)

This cell demonstrates how an external client would call the MCP server endpoints to run tools and read resources.

In [19]:

import requests, json
BASE = 'http://127.0.0.1:8000'

print('Initialize-like call (list tools/resources)')
print(requests.get(f'{BASE}/list_tools').json())
print(requests.get(f'{BASE}/list_resources').json())

# Call search_flights via MCP
req = {'name':'search_flights','arguments':{'origin':'SFO','destination':'Paris','depart_date':'2025-10-10','passengers':1}}
r = requests.post(f'{BASE}/call_tool', json=req)
print('\nsearch_flights result:\n', json.dumps(r.json(), indent=2))

# Read resources after call
print('\nResources after call:')
print(requests.get(f'{BASE}/list_resources').json())

Initialize-like call (list tools/resources)
INFO:     127.0.0.1:40138 - "GET /list_tools HTTP/1.1" 200 OK
[{'name': 'search_flights', 'description': 'Search flights via web + LLM', 'inputSchema': {'origin': 'string', 'destination': 'string', 'depart_date': 'string', 'return_date': 'string?', 'passengers': 'int?'}}, {'name': 'search_hotels', 'description': 'Search hotels via web + LLM', 'inputSchema': {'destination': 'string', 'checkin': 'string', 'checkout': 'string', 'guests': 'int?'}}, {'name': 'plan_itinerary', 'description': 'Plan itinerary via web + LLM', 'inputSchema': {'destination': 'string', 'days': 'int', 'interests': 'array?'}}]
INFO:     127.0.0.1:40148 - "GET /list_resources HTTP/1.1" 200 OK
[{'uri': 'travel://flights/search', 'name': 'Flight Search Results', 'mimeType': 'application/json', 'description': ''}, {'uri': 'travel://hotels/search', 'name': 'Hotel Search Results', 'mimeType': 'application/json', 'description': ''}, {'uri': 'travel://itineraries/latest', 'name': 